In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append('../')

In [2]:
import gc
from typing import Callable, List, Tuple

import pandas as pd
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import svds
import time
from tqdm import tqdm

from src.preprocess import add_time_idx
from src.prepare_data import train_val_test_split, get_users_history
from src.SVD import SVD
from src.unbiased_metrics import get_metrics, hr, mrr, ndcg

## Load Data

In [3]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_table('../data/ml-1m.dat', names = columns, sep = "::", encoding = "latin1", engine='python')

In [4]:
train, val_1, val_2, test, train_dict, test_dict = train_val_test_split(data, filter_negative=False)

In [5]:
test_users_history = get_users_history(test, train_dict, test_dict)

47862it [2:44:03,  4.86it/s]


In [6]:
val_users_history_1 = get_users_history(val_1, train_dict, test_dict)

23931it [39:42, 10.04it/s]


In [7]:
val_users_history_2 = get_users_history(val_2, train_dict, test_dict)

23931it [42:35,  9.37it/s]


In [8]:
train.to_csv('train.csv', index=False)
val_1.to_csv('val_1.csv', index=False)
val_2.to_csv('val_2.csv', index=False)
test.to_csv('test.csv', index=False)
test_users_history.to_csv('test_users_history.csv', index=False)
val_users_history_1.to_csv('val_users_history_1.csv', index=False)
val_users_history_2.to_csv('val_users_history_2.csv', index=False)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val_2 = pd.read_csv('val_2.csv')
test_users_history = pd.read_csv('test_users_history.csv')
val_users_history_2 = pd.read_csv('val_users_history_2.csv')

# Hyper parameters selection 

In [9]:
def grid_search(param_list: list, 
                train: pd.DataFrame, 
                val_history: pd.DataFrame, 
                val: pd.DataFrame) -> Tuple[list, float]:
    best_params = param_list[0]
    max_ndcg = -1
    for params in param_list:
        svd = SVD(**params)
        svd.fit(train)
        scores = svd.predict_folding_in(val_history)
        preds = svd.get_top_k(scores, user_col = 'test_user_idx')      
        
        preds = preds.merge(val, on='test_user_idx', how='inner') 
        preds = preds[preds['rating'] >= 3.5]
        
        cur_ndcg = ndcg(preds)
        print(f'params = {params}')
        print(f'ndcg = {cur_ndcg}')
        
        if cur_ndcg > max_ndcg:
            max_ndcg = cur_ndcg
            best_params = params
    return best_params, max_ndcg

In [10]:
param_grid = {'alpha': np.linspace(0.1, 0.8, 8), 'n_factors': np.linspace(50, 500, 10).astype(int)}
param_list = list(ParameterGrid(param_grid))

In [11]:
best_params, max_ndcg = grid_search(param_list, train, val_users_history_1, val_1)

100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


params = {'alpha': 0.1, 'n_factors': 50}
ndcg = 0.032193


100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


params = {'alpha': 0.1, 'n_factors': 100}
ndcg = 0.034343


100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


...

params = {'alpha': 0.8, 'n_factors': 400}
ndcg = 0.023651


100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


params = {'alpha': 0.8, 'n_factors': 450}
ndcg = 0.022186


100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


params = {'alpha': 0.8, 'n_factors': 500}
ndcg = 0.021361


In [12]:
best_params, max_ndcg

({'alpha': 0.4, 'n_factors': 150}, 0.037926)

In [4]:
best_params = {'alpha': 0.4, 'n_factors': 150}

In [5]:
svd = SVD(**best_params)
svd.fit(train)
scores_val = svd.predict_folding_in(val_users_history_2)
scores_test = svd.predict_folding_in(test_users_history)

preds_val = svd.get_top_k(scores_val, user_col='test_user_idx') 
preds_test = svd.get_top_k(scores_test, user_col='test_user_idx')

preds_val = preds_val.merge(val_2, on='test_user_idx', how='inner')
preds_test = preds_test.merge(test, on='test_user_idx', how='inner') 

metrics_df, beta = get_metrics(preds_test, preds_val)

100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


In [6]:
metrics_df

,type,HR,MRR,nDCG
0,Biased,0.088995,0.022310,0.037369
1,Unbiased,0.147800,0.081114,0.079055
2,Unbiased_feedback_sampling,0.532370,0.178634,0.047297


In [7]:
beta

0.06454922265522797

In [8]:
preds_test_pos = preds_test[preds_test['rating'] >= 3.5]

In [9]:
hr(preds_test_pos, beta=beta, sample_feedback=True, return_confidence_interval=True)

(0.53237, 0.0012565594698326077)

In [10]:
mrr(preds_test_pos, beta=beta, sample_feedback=True, return_confidence_interval=True)

(0.178634, 0.0007327384206406162)

In [11]:
ndcg(preds_test_pos, beta=beta, sample_feedback=True, return_confidence_interval=True)

(0.047297, 3.293627487450934e-05)